### Data collection for various metrics:

For Ours:
Data Collection for Safety and Stability Metrics (Ignore the efficiency metrics even though they are displayed). 

Collection performed in 2 stages

    A. Data collection for Safety and CAV (Controller Acceleration Variation) 
    B. Data collevtion for WAR (Wave Attenuation Ratio)

### Notes:

- Desired velocity change for FS May need monitoring (May not be set for all possible lengths)
- PI can have failures, so have to turn ON render and manually check
- Generated data will be saved in the folder `test_time_rollout`

- At test time: The inflow should be fixed and be same as the classic controllers 
- Sim step: At training, was set to 0.5. At test, set to 0.1

## When analyzing the csv files
- For RL shocks start at 7400 and end at 11000
- To make it fair, for classic controller the rollouts goes upto 11000 with shocks that come up at 7400 (for some reason, the warmup value makes the horizon longer but does not push the shock start time back.)


### Additional Notes
- When sampling shock vehicles, sometimes a vehicle that has exited the network may be sampled and cause an error. Redo the experiment in that case.

In [ ]:
NUM_ROLLOUTS = 10 # There is more variation in the sampling

# Just change the av_frac and rerun the experiments
AV_FRAC =  0.05 #0.05, 0.2, 0.4, 0.6

### 1. (100%) IDM 

In [ ]:
# idm (although av_frac is not used, must be supplied)
!python classic.py --method idm --gen_emission --num_rollouts $NUM_ROLLOUTS\
--render --shock --av_frac 1.0 --sim_step 0.5 --shock_start_time 500 \
--shock_end_time 1220 --horizon 1220

In [ ]:
!python eval_metrics.py --method idm --start_time 500 --end_time 1220 --sim_step 0.5

### 2. Multiple Vehicle Control at % deployments

### FS

In [ ]:
!python classic.py --method fs --gen_emission --num_rollouts $NUM_ROLLOUTS \
--render --shock --av_frac $AV_FRAC --sim_step 0.5 --shock_start_time 500 \
--shock_end_time 1220 --horizon 1220

In [ ]:
!python eval_metrics.py --method fs --start_time 500 --end_time 1220 --sim_step 0.5

### PIwS

In [ ]:
# piws
!python classic.py --method piws --gen_emission --num_rollouts $NUM_ROLLOUTS\
--shock --render --av_frac $AV_FRAC --sim_step 0.5 --shock_start_time 500 \
--shock_end_time 1220 --horizon 1220

In [ ]:
!python eval_metrics.py --method pi --start_time 500 --end_time 1220 --sim_step 0.5

### BCM

In [ ]:
# bcm 
!python classic.py --method bcm --num_rollouts $NUM_ROLLOUTS \
--gen_emission --shock --render --av_frac $AV_FRAC --sim_step 0.5 --shock_start_time 500 \
--shock_end_time 1220 --horizon 1220

In [ ]:
!python eval_metrics.py --method bcm --start_time 500 --end_time 1220 --sim_step 0.5

### LACC

In [ ]:
# lacc
!python classic.py --method lacc --num_rollouts $NUM_ROLLOUTS\
--gen_emission --shock --render --av_frac $AV_FRAC --sim_step 0.5 --shock_start_time 500 \
--shock_end_time 1220 --horizon 1220

In [ ]:
!python eval_metrics.py --method lacc --start_time 500 --end_time 1220 --sim_step 0.5

### Notes: 
- Some things are different for Vinitsky and Ours
- Sim steps is 0.5 so although horizon is 11000. It ends at 5500. These are longer than what is seen at training (1750).
- Shocks normally begin at 7400 and end at 11000 (for 0.1 timesteps = 360 seconds as sim_step) but here (at 0.5 sim_steps):

1. begin at 500 and end at 1240 i.e., they run for 720 steps (360 seconds)

- Lokks like not a single change in eval_metrics.
- There is no warmup at test time (same for classic controllers)

### Vinitsky
- Now we have different policies for each av_frac
- Always monitor, can unexpectedly quit

### (5%)

In [ ]:
# copy the exp_configs to the test_rllib.py folder 
# For some reason, this replaces its own file so have to be done this way
for _ in range(NUM_ROLLOUTS):
    !python test_rllib.py ./Vinitsky_et_al/Trained_policies/5_percent/PPO_BottleneckDesiredVelocityEnv-v0_ae80d23e_2024-02-20_12-15-449lzgp6ar 250 \
    --method vinitsky --num_rollouts 1 --gen_emission --shock --render --sim_step 0.5 --shock_start_time 500 \
    --shock_end_time 1220 --horizon 1220
    

In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5

### (20%)

In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5

### (40%)

In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5

### (60%)

In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5

### Ours
- Now we have different policies for each av_frac
- At test time, set min_gap to 5

### (5%)

In [ ]:
###### SAFETY + STABILITY ######
# First change the avfrac in the exp_configs and copy the exp_configs to the test_rllib.py folder 
# For some reason, this replaces its own file so have to be done this way
for _ in range(NUM_ROLLOUTS):
    !python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareBottleneckEnv-v1_6212be34_2024-02-20_19-30-152ok87cho 354\
    --method ours --num_rollouts 1 --gen_emission --shock --render --min_gap 10.0 --sim_step 0.5 --shock_start_time 500 \
    --shock_end_time 1220 --horizon 1220 


In [ ]:
!python eval_metrics.py --method ours --start_time 500 --end_time 1220 --sim_step 0.5

In [ ]:
###### Efficiency ######
# Make sure to turn on the zero acceleration behavior at test time


### (20%)

In [ ]:
###### SAFETY + STABILITY ######


In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5

### (40%)

In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5